In [2]:
#imports
from google.cloud import speech
import io

In [34]:
tts(text_to_read)
init_google()
record_file()
text_to_read = transcribe()
dialogmanager()

Coronabot: Guten Tag! Ich bin ein Chatbot, der Sie über Impfzahlen in Deutscchland in formiert. Fragen Sie nach Impfzahlen für die verschiedenen Impfstoffe für die Bundesländer
Transcript 0: moderner in Höchst


AttributeError: 'NoneType' object has no attribute 'lower'

In [31]:
init_google()
record_file()
text_to_read = transcribe()
#tts(text_to_read)

Transcript 0: moderner in Hessen


In [ ]:
def welcome():
        

In [3]:
#initiates the recording and the the transcription 
def speech_input():
    record_file()
    text_to_read = transcribe()
    return text_to_read

In [4]:
#imports
import os
import sounddevice as sd
import soundfile as sf
import numpy as np
from scipy.io.wavfile import write
sr = 16000  # Sample rate
seconds = 3  # Duration of recording
filename = 'myfile.wav'

#recording
def record_file():
    data = sd.rec(int(seconds * sr), samplerate=sr, channels=1)
    sd.wait()  # Wait until recording is finished
    # Convert `data` to 16 bit integers:
    y = (np.iinfo(np.int16).max * (data/np.abs(data).max())).astype(np.int16) 
    write(filename, sr, y)

In [5]:
#google stt credentials
def  init_google():
    credentials='C:\\Users\\elisa\\Seminare_Python\\S_dialogsystem\\dialogsystem-1627986561247-c6ec7c426248.json'
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=credentials

In [6]:
#print the transcription with google
def transcribe():
    client = speech.SpeechClient()
    with io.open(filename, "rb") as audio_file:
        content = audio_file.read()
    audio = speech.RecognitionAudio(content = content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        language_code="de-DE",
    )
    response = client.recognize(config=config, audio=audio)
    for result in response.results:
        for index, alternative in enumerate(result.alternatives):
            print("Transcript {}: {}".format(index, alternative.transcript))
            return alternative.transcript

In [24]:
import pyttsx3
def tts(text_to_read):
    text_to_read = 'Guten Tag! Ich bin ein Chatbot, der Sie über Impfzahlen in Deutscchland in formiert. Fragen Sie nach Impfzahlen für die verschiedenen Impfstoffe für die Bundesländer'
    engine = pyttsx3.init()
    engine.setProperty('voice', 'german')
    engine.setProperty('rate', 200)
    engine.say(text_to_read)
    engine.runAndWait()   
    print('Coronabot: '+text_to_read)

    

In [8]:
#defines states in API
states_api = {'baden':'BW', 'bayern':'BY', 'berlin':'BE', 'brandenburg':'BB', 'bremen':'HB', 'hamburg': 'HH', 'hessen': 'HE', 'mecklenburg':'MV', 'niedersachsen':'NI', 'nordrhein':'NW', 'rheinland':'RP', 'saarland':'SL', 'sachsen':'SN', 'anhalt':'SA', 'schleswig':'SH', 'thüringen':'TH'}
#formatting for response
states_back = {'BW':'Baden-Württemberg', 'BY':'Bayern', 'BE':'Berlin', 'BB':'Brandenburg', 'HB':'Bremen', 'HH':'Hamburg', 'HE':'Hessen', 'MV':'Mecklenburg-Vorpommern', 'NI':'Niedersachen', 'NW':'Nordrhein-Westfalen', 'SL':'Saarland', 'SN':'Sachsen', 'SA':'Sachsen-Anhalt', 'SH':'Schleswig-Holstein', 'TH':'Thüringen'}
#defines vaccine names in API
vaccines_api = {'biontech':'biontech', 'biontek':'biontech', 'biontec':'biontech', 'moderna':'moderna', 'astraZeneca':'astraZeneca', 'astra':'astraZeneca', 'zeneca':'astraZeneca', 'delta':'delta'}
#formatting vaccines for respionce
vaccines_back = {'biontech':'Biontech', 'moderna':'Moderna', 'astraZeneca':'AstraZeneca', 'delta':'Delta'}

import requests

def dialogmanager():
    input_s = do_input() #do_input() wird aufgerufen, um sich input zu holen
    input_s = normalize(input_s)
    while input_s and input_s != 'fertig': #wenn input-string 'fertig'  ist, wird Schleife nicht ausgeführt
        semantics = semantic(input_s) #funktion gibt etwas zurück
        result = data(semantics)
        out_string = output(semantics, result)
        tts(out_string)
        input_s = do_input()

#everything as lower case        
def normalize(in_s):
    return in_s.lower()        
        
def do_input():
    return speech_input()

#finds states and vaccines in input
def semantic(input_s):
    semantics = {'state':'', 'vaccine':''} #data structure
    for key in states_api.keys():
        if key in input_s:
            semantics['state'] = states_api[key]
    for key in vaccines_api.keys():
        if key in input_s:
            semantics['vaccine'] = vaccines_api[key]
    return semantics

#data from RKI API
vaccinations = requests.get('https://api.corona-zahlen.org/vaccinations')
def data(semantics):
    s = semantics['state']
    v = semantics['vaccine']
    if s: #state given
        if s != 'DE':
            if v: #and vaccine given
                vacc_number = vaccinations.json()["data"]["states"][s]["vaccination"][v] #
            else: #all vaccines for state
                vacc_number = vaccinations.json()["data"]["states"][s]["vaccinated"]
        else:
            if v: #and vaccine given
                vacc_number = vaccinations.json()["data"]["vaccination"][v]
            else: #all vaccines in Germany
                vacc_number = vaccinations.json()["data"]["vaccinated"]
    else: #no state
        if v: #but vaccine
            vacc_number = vaccinations.json()["data"]["vaccination"][v]
        else: #nothing given
            vacc_number = None
    return vacc_number

def output(semantics, results): #fügt gut oder schlecht ein
    ret = ''
    s = semantics['state']
    v = semantics['vaccine']
    if s: #if state is given
        s = states_back[semantics['state']]
        if v: #vaccine also given
            v = vaccines_back[semantics['vaccine']]
            ret = 'Die Impfungen für {} mit {} sind {}'.format(s, v, results) #formatting
        else: #all vaccines for states
            ret = 'Die Impfungen für {} sind {}'.format(s, results)
    else: #no state
        if v: #but vaccine
            v = vaccines_back[semantics['vaccine']]
            ret = 'Die Impfungen in Deutschland mit {} sind {}'.format(v, results)
        else: #nothing given
            ret = 'Diese Frage kann ich leider nicht beantworten'
    return ret
        
def do_error():
    return 'Bitte wählen Sie ein Bundesland um die Coronazahlen zu erhalten'